In [1]:
import pandas as pd

df = pd.read_csv('DataCoSupplyChainDataset.csv', encoding='ISO-8859-1')

print("Data Loaded Successfully!")
df.head()

Data Loaded Successfully!


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# 1. CLEAN UP COLUMN NAMES

df.columns = [col.lower().replace(' ', '_').replace('(', '').replace(')', '') for col in df.columns]

# 2. CREATE MOCK CARRIERS 

carriers = ['Uber Freight', 'Speedy Logistics', 'US Xpress', 'Yellow Freight', 'Ready Trans']
np.random.seed(42) 
df['carrier_name'] = np.random.choice(carriers, size=len(df))

# 3. FIX DATES

df['order_date'] = pd.to_datetime(df['order_date_dateorders'])
df['shipping_date'] = pd.to_datetime(df['shipping_date_dateorders'])

# 4. CREATE YOUR KPIs 

df['is_late'] = df['delivery_status'].apply(lambda x: 1 if 'Late' in x else 0)

df['delay_days'] = df['days_for_shipping_real'] - df['days_for_shipment_scheduled']

# 5. SAVE TO DATABASE (SQL)

conn = sqlite3.connect('supply_chain.db')
df.to_sql('shipments', conn, if_exists='replace', index=False)
conn.close()

print("Data cleaning complete. Database 'supply_chain.db' created successfully!")
df[['carrier_name', 'is_late', 'delay_days', 'order_date']].head()

Data cleaning complete. Database 'supply_chain.db' created successfully!


,carrier_name,is_late,delay_days,order_date
0,Yellow Freight,0,-1,2018-01-31 22:56:00
1,Ready Trans,1,1,2018-01-13 12:27:00
2,US Xpress,0,0,2018-01-13 12:06:00
3,Ready Trans,0,-1,2018-01-13 11:45:00
4,Ready Trans,0,-2,2018-01-13 11:24:00


In [3]:
df.to_csv('final_shipment_data.csv', index=False)